In [ ]:
import logging
from filecmp import dircmp
from os import listdir
import os
import filecmp
from dotenv import load_dotenv
import matplotlib as plt
import pandas as pd
import numpy as np
from shapely import wkt
import shapely
from sqlalchemy import create_engine
from shapely.geometry import shape
from shapely.ops import unary_union
from geopandas import GeoDataFrame
import geopandas as gpd
from datetime import datetime
import networkx as nx
from shapely.geometry import MultiPolygon
import datetime as dt
from pystac_client import Client
from holoviews import opts
from intake import open_catalog

load_dotenv()

In [ ]:
catfeux = open_catalog(f'{os.getenv("PROJECT_PATH")}Fire_Detection_Data_Quality.yaml')

date_start=datetime(2024,1,1)
date_end=datetime(2024,3,31)

table_source='surfaces_brulees_brute_control'
sql = f"""SELECT *
FROM feux_cq.{table_source} si
WHERE si.date_ >= '{pd.to_datetime(date_start).strftime('%Y-%m-%d')}' AND si.date_ <= '{pd.to_datetime(date_end).strftime('%Y-%m-%d')}'
"""

dataCatalog = getattr(catfeux, table_source)(sql_expr=sql)
surfdetect_control = dataCatalog.read()

tile_sentinel=catfeux.tile_sentinel2_line_UTM.read()
tile_sentinel=tile_sentinel.to_crs(epsg=4326)

nc_limits=catfeux.nc_limits.read()
nc_limits=nc_limits.to_crs(epsg=4326)

In [ ]:
from shapely.geometry import Polygon, mapping
from bokeh.models import ColumnDataSource, HoverTool, LogColorMapper

def linestring_to_polygon(gdf):
    polygons = []
    
    for index, row in gdf.iterrows():
        all_coords = mapping(row['geometry'])['coordinates']
        lats = [x[1] for x in all_coords]
        lons = [x[0] for x in all_coords]
        
        polyg = Polygon(zip(lons, lats))
        polygons.append(polyg)
    new_gdf = gpd.GeoDataFrame(geometry=polygons, crs=gdf.crs)
    
    return new_gdf

test = linestring_to_polygon(tile_sentinel)
tile_sentinel['geometry']=test['geometry']
tile_sentinel['Name']='L2A_T'+tile_sentinel['Name']

In [ ]:
centroid = (tile_sentinel.centroid)
centroid_tuile = pd.DataFrame(centroid)

centroid_tuile['x']=centroid.x
centroid_tuile['y']=centroid.y
centroid_tuile['nom']=tile_sentinel['Name']

In [ ]:
def getPolyCoords(row, geom, coord_type):
    """Returns the coordinates ('x' or 'y') of edges of a Polygon exterior"""
    exterior = row[geom].exterior

    if coord_type == 'x':
        return list( exterior.coords.xy[0] )
    elif coord_type == 'y':
        return list( exterior.coords.xy[1] )


tile_sentinel['x'] = tile_sentinel.apply(getPolyCoords, geom='geometry', coord_type='x', axis=1)
tile_sentinel['y'] = tile_sentinel.apply(getPolyCoords, geom='geometry', coord_type='y', axis=1)

m_df = tile_sentinel.drop('geometry', axis=1).copy()
tuile = ColumnDataSource(m_df)

nc_limits=nc_limits.explode()
nc_limits['x'] = nc_limits.apply(getPolyCoords, geom='shape', coord_type='x', axis=1)
nc_limits['y'] = nc_limits.apply(getPolyCoords, geom='shape', coord_type='y', axis=1)

m_df_2 = nc_limits.drop('shape', axis=1).copy()
nc = ColumnDataSource(m_df_2)

## Find detection type : "Mono detection" or "pluri detection"

In [ ]:
def find_intersecting_id(row, gdf):

    possible_matches_index = list(gdf.sindex.intersection(row['geometry'].bounds))
    possible_matches = gdf.iloc[possible_matches_index]
    precise_matches = possible_matches[possible_matches.geometry.intersects(row['geometry'])]
    intersecting_ids = precise_matches['surface_id_h3'].tolist()
    intersecting_ids = [id_ for id_ in intersecting_ids if id_ != row['surface_id_h3']]
    return intersecting_ids

In [ ]:
def mesure_totale(df):
    tot_nb=len(df) #number total of detection 
    tot_surf=df.dissolve() ## total number of detected area (ha)
    tot_surf=pd.Series(tot_surf.area/10000)
    tot_surf=tot_surf.reset_index(drop=True)
    tot_surf=tot_surf[0]
    
    tot_surf_tile=df.dissolve(by='nom')
    tot_surf_tile=pd.DataFrame(tot_surf_tile.area/10000)
    tot_surf_tile['nom']=tot_surf_tile.index
    tot_surf_tile=tot_surf_tile.reset_index(drop=True)

    return(tot_surf,tot_nb,tot_surf_tile)

def mesure_pluri_detection(df):
    pluri_detection_list=df[df['groupe_id'].notna()]
    pluri_detection_surface=pluri_detection_list.dissolve() ## number of pluri detected detected area (ha)
    pluri_detection_surface=pd.Series(pluri_detection_surface.area/10000)
    pluri_detection_surface=pluri_detection_surface.reset_index(drop=True)
    pluri_detection_surface=pluri_detection_surface[0]

    pluri_detection_group=pluri_detection_list['groupe_id'].nunique() ## number of group
    pluri_tile_number = pd.DataFrame(pluri_detection_list["nom"].value_counts())
    
    pluri_tile_surface=pluri_detection_list.dissolve(by='nom')
    pluri_tile_surface=pd.DataFrame(pluri_tile_surface.area/10000)
    pluri_tile_surface['nom']=pluri_tile_surface.index
    pluri_tile_surface=pluri_tile_surface.reset_index(drop=True)
    
    return(pluri_tile_surface,pluri_tile_number,pluri_detection_group,pluri_detection_surface)

def mesure_mono_detection(df):
    mono_detection_list=df[df['groupe_id'].isna()]
    mono_detection_surface=mono_detection_list['surface'].sum() ## number of mono detected detected area (ha)
    mono_detection_group=mono_detection_list['groupe_id'].isna().sum() ## number of mono detected polygons

    mono_tile_number = pd.DataFrame(mono_detection_list["nom"].value_counts()) ## number of detection per tiles
    mono_tile_surface = mono_detection_list.groupby('nom')['surface'].sum().reset_index() ## sum of burned area detected per tile
    
    return(mono_tile_surface,mono_tile_number,mono_detection_group,mono_detection_surface)

In [ ]:
def try_multiple_date_formats(date_str, formats):
    for fmt in formats:
        try:
            return pd.to_datetime(date_str, format=fmt)
        except ValueError:
            continue
    return pd.NaT 

In [ ]:
def stac_search(date_start,date_end):
    from pystac_client import Client

    catalog = Client.open("https://earth-search.aws.element84.com/v1")
    query = catalog.search(
        collections=["sentinel-2-l2a"],datetime=(date_start).strftime('%Y-%m-%d')+'/'+(date_end).strftime('%Y-%m-%d'), bbox=[163.362, -22.76, 168.223, -19.479],        fields={"include": ["properties.grid:code", "properties.datetime", "properties.eo:cloud_cover", "assets.thumbnail.href"], "exclude": []}
    )

    items = list(query.items())
    stac_json = query.item_collection_as_dict()

    gdf = gpd.GeoDataFrame.from_features(stac_json, "epsg:4326")
    thumbnails = [item.assets['thumbnail'].href for item in items]

    df = gdf.rename(columns={
        'grid:code': 'nom',
        'datetime': 'date_',
        'eo:cloud_cover': 'Cloud_Cover',
        'thumbnail.href': 'thumbnail'
    })

    df['nom'] = [x[5:] for x in df['nom']]
    df['nom']='L2A_T'+df['nom'] 

    df=df.reset_index(drop=True)
    date_formats = ['%Y-%m-%dT%H:%M:%SZ', '%Y-%m-%d %H:%M:%S.%f%z','%Y-%m-%dT%H:%M:%S.%fZ']
    df['date_'] = df['date_'].apply(try_multiple_date_formats, formats=date_formats)
    df['date_'] = df['date_'].dt.strftime('%Y-%m-%d')
    df['date_'] = pd.to_datetime(df['date_'])
    
    df['thumbnail_url'] = thumbnails
    df = df.sort_values(by='date_', ascending=True)

    return(df)

In [ ]:
def apply_formatter(plot, element):
    formatter = DatetimeTickFormatter(
        microseconds=["%Y-%m-%d"],
        milliseconds=["%Y-%m-%d"],
        seconds=["%Y-%m-%d"],
        minsec=["%Y-%m-%d"],
        minutes=["%Y-%m-%d"],
        hourmin=["%Y-%m-%d"],
        hours=["%Y-%m-%d"],
        days=["%Y-%m-%d"],
        months=["%Y-%m-%d"],
        years=["%Y-%m-%d"],
    )
    plot.handles['xaxis'].formatter = formatter

In [ ]:
import panel as pn
import numpy as np
from bokeh.models import HoverTool
from bokeh.palettes import RdYlBu11 as palette
from bokeh.models import LogColorMapper
from bokeh.tile_providers import OSM, get_provider
from bokeh.plotting import figure
import datetime as dt
from odc.stac import configure_rio, stac_load
import matplotlib.pyplot as plt
import holoviews as hv
from bokeh.models import ColumnDataSource, DatetimeTickFormatter
from bokeh.models.formatters import DatetimeTickFormatter
from odc.stac import configure_rio, stac_load
import requests
from PIL import Image
from io import BytesIO
import hvplot.pandas 

pn.extension()
pn.extension('tabulator')

stylesheet = """
.tabulator-cell {
    font-size: 20px;
}
"""

custom_style = {
    'background': '#f89424',
    'border': '1px solid black',
    'padding': '10px',
    'box-shadow': '5px 5px 5px #bcbcbc'
}
    
def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    return ['background-color: f89424' if v else '' for v in is_max]

tile_bouton = pn.widgets.RadioButtonGroup(options=['L2A_T58KCC','L2A_T58KCD','L2A_T58KDB','L2A_T58KDC','L2A_T58KEA','L2A_T58KEB','L2A_T58KEC',
            'L2A_T58KFA','L2A_T58KFB','L2A_T58KFC','L2A_T58KGA','L2A_T58KGB','L2A_T58KGC','L2A_T58KGV','L2A_T58KHB'],align='center',stylesheets=[stylesheet],
            button_type='warning',button_style='outline',name='Choose a tile')

### PAGE 1 #########
############ table

def maj_table(date_range):
    hv.extension('bokeh')
    global stac_search_results
    global df
    
    surfdetect_control['date_']=pd.to_datetime(surfdetect_control['date_'])
    df=surfdetect_control.loc[(surfdetect_control['date_'] >= date_range[0]) & (surfdetect_control['date_'] <= date_range[1])]

    df['nom']=df['nom'].apply(lambda x: x[20:])
    df['groupe_id'] = np.nan
    stac_search_results=stac_search(df['date_'].min(),df['date_'].max())

    G = nx.Graph()

    for index, row in df.iterrows():
        intersecting_ids = find_intersecting_id(row, df)
        for id_ in intersecting_ids:
            G.add_edge(row['surface_id_h3'], id_)

    groupes = list(nx.connected_components(G))

    for groupe_id, groupe in enumerate(groupes):
        for id_ in groupe:
            df.loc[df['surface_id_h3'] == id_, 'groupe_id'] = groupe_id

    pluri_tile_surface,pluri_tile_number,pluri_detection_group,pluri_detection_surface=mesure_pluri_detection(df)
    mono_tile_surface,mono_tile_number,mono_detection_group,mono_detection_surface=mesure_mono_detection(df)
    tot_surf,nb_tot,tot_surf_tile=mesure_totale(df)

    info_surfaces = pd.merge(mono_tile_number, mono_tile_surface, on='nom', how='outer')
    info_surfaces = pd.merge(info_surfaces, pluri_tile_number, on='nom', how='outer')
    info_surfaces = pd.merge(info_surfaces, pluri_tile_surface, on='nom', how='outer')

    info_surfaces=info_surfaces.rename(columns={'nom':'Tile name','count_x':'Number of mono detection','surface':'Sum of mono detected area','count_y':'Number of pluri detection',0:'Sum of pluri detected area'})
    info_surfaces=info_surfaces.round(2)

    table = pn.widgets.Tabulator(info_surfaces, name="Informations à l'échelle des tuiles Sentinel-2",header_align='center', show_index=False,
                stylesheets=[stylesheet])
    table.style.apply(highlight_max)
#################################    
    map = figure(width=800,title="Carte des surfaces brûlées estimées par tuiles pour les dates sélectionnées",x_range=(162.5,169.5))

    map.patches('x', 'y', source=nc,
        fill_alpha=1, line_color="black", line_width=0.2)    ## plot of new caledonia land

    line=map.patches('x', 'y', source=tuile, ## plot of tiles limits
            fill_alpha=0, line_color="black", line_width=1)
    
    tt=pd.merge(centroid_tuile, tot_surf_tile, on='nom', how='left')

    source_tt = ColumnDataSource(data={'x': tt['x'], 'y': tt['y'], 'surfaces': tt['0_y'], 'name': tt['nom']})
    facteur_de_reduction = 0.1
    sizes_reduites = [size * facteur_de_reduction for size in tt['0_y']]

    source_tt.data['sizes'] = sizes_reduites

    circles=map.circle('x', 'y', source=source_tt, size='sizes' ,
            fill_alpha=1,fill_color="orange", line_color="black", line_width=1)

    tooltip = HoverTool(renderers=[circles])  
    tooltip.tooltips = [
        ('Tuile', '@name'),
        ('Surface', '@surfaces')  
    ]
    map.add_tools(tooltip)

    return(table,map,nb_tot,tot_surf,mono_detection_group,pluri_detection_group,mono_detection_surface,pluri_detection_surface)

############################

def maj_graphic(date_range,choix):
    global stac_search_results
    global df

    date_range = pd.date_range(start=df['date_'].min(), end=df['date_'].max())
    full_date_series = pd.DataFrame(date_range.strftime('%Y-%m-%d'), columns=['date_'])
    full_date_series['date_']=pd.to_datetime(full_date_series['date_'])

    df_tiles = df.groupby(['date_', 'nom']).size().reset_index(name='nombre_occurrences')

    df_tiles = df_tiles[df_tiles['nom'] == choix]
    
    df_tiles = pd.merge(full_date_series, df_tiles, on='date_', how='left')
    df_tiles['nom'] = df_tiles['nom'].fillna(choix)

    df_cloud_cover=stac_search_results[stac_search_results['nom'] == choix]
    df_tot= pd.merge(df_tiles, df_cloud_cover, on='date_', how='left')

    tile = hv.Bars(df_tot,  ('date_','Date'), ('nombre_occurrences','Nombre de détection')).opts(color='orange',tools=['hover']).opts(color='orange')
    cc_fig=df_tot.hvplot(x='date_', y='Cloud_Cover', kind='scatter').opts(color='black', marker='s', size=30) 

    combined = (tile + cc_fig).cols(1)
    date_formatter = '%Y-%m-%d' 
    combined.opts(
        opts.Scatter(height=400, width=1800, xrotation=45, responsive=True,title='Evolution de la couverture nuageuse (%)',hooks=[apply_formatter],shared_axes=True),
        opts.Bars(height=600, width=1800, xrotation=45, responsive=True,title="Nombre de détection par jour",hooks=[apply_formatter],shared_axes=True),
        opts.Layout(shared_axes=True))

    image_elements = []
    for _, row in df_cloud_cover.iterrows():
        url = row['thumbnail_url']
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))
        img_array = np.array(img)
        image_elements.append(hv.RGB(img_array).opts(title=f"Date: {row['date_'].date()}, Cloud: {row['Cloud_Cover']}%"))

    grid = hv.Layout(image_elements).opts(opts.RGB(width=500, height=500, xaxis=None, yaxis=None)).cols(3)
    grid = hv.Layout(grid).opts(width=1200,height=600)

    return combined,grid

total_detection=pn.indicators.Number(name='Totale détection', value=0, format='{value}',colors=[(0,'blue')])
surface_total=pn.indicators.Number(name='Surface totale estimée (ha)', value=0, format='{value}',colors=[(0,'blue')])
mono_detection_group=pn.indicators.Number(name='Nombre de Mono détection', value=0, format='{value}',colors=[(0,'red')])
pluri_detection_group=pn.indicators.Number(name='Nombre de Pluri détections', value=0, format='{value}',colors=[(0,'green')])
mono_detection_surface=pn.indicators.Number(name='Surface (ha) Mono détection', value=0, format='{value}',colors=[(0,'red')])
pluri_detection_surface=pn.indicators.Number(name='Surface (ha) Pluri détections', value=0, format='{value}',colors=[(0,'green')])

table_map_container = pn.Row()  
graphic_container = pn.Column() 
interface_1_container = pn.Column()

def update_interface_1(event):
    global table_map_container
    
    table, map, nb_tot, tot_surf, mono_nb, pluri_nb, mono_surf, pluri_surf = maj_table(datetime_range_picker.value) 
    mono_detection_group.value = mono_nb
    pluri_detection_group.value = pluri_nb
    
    mono_detection_surface.value = mono_surf
    pluri_detection_surface.value = pluri_surf

    total_detection.value = nb_tot
    surface_total.value = tot_surf
    
    table_map_container[:] = [table, map]
    interface_1_container[:] = [table_map_container, tile_bouton]
    
    if interface_1_container not in main:
        main.append(interface_1_container)
    if graphic_container not in main:
        main.append(graphic_container)

def update_interface_2(event):
    global graphic_container
    
    choix = event.new
    fig, image = maj_graphic(datetime_range_picker.value, tile_bouton.value)
    graphic_container[:] = [fig, image]    

datetime_range_picker = pn.widgets.DatetimeRangePicker(name='Select your Date Range', start=dt.datetime(2023, 1, 1), end=dt.datetime(2024, 12, 31))
datetime_range_picker.param.watch(update_interface_1, 'value')

tile_bouton.param.watch(update_interface_2, 'value')

sidebar = pn.Column(datetime_range_picker,"# Indicateurs Globaux", total_detection,surface_total,mono_detection_group, mono_detection_surface,pluri_detection_group,pluri_detection_surface)
main = pn.Column("## Step 1 : Selectionner un intervalle de date pour voir les données et indicateurs globaux. \n ## Step 2 : Choisir une ZAE à observer") 

template =pn.template.FastListTemplate(
    site="Panel", header_background ='#f89424',title="Dashboard Contrôle des surfaces brûlées en sortie en chaîne",logo="https://neotech.nc/wp-content/uploads/2023/10/logo_oeil_quadri-254x300.jpeg.webp",sidebar=[sidebar],main=[main])

template.servable()

## panel serve 03_Dashboard_controle_V2.ipynb